# 유튜브 실시간 인기영상 타이틀 분석하기
> Jupyter notebook에서 크롤링을 한다음 파일로 저장 colab에서 분석해 보겠다.

In [3]:
import pandas as pd
import numpy as np
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori
from mlxtend.frequent_patterns import association_rules
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
dataset = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Lec4_BigData/4.apriori/data/youtubedata.csv')
dataset.head()

,Unnamed: 0,title,video_url
0,0,영화 '발신제한' 개봉 첫 주말 박스오피스 정상 올랐다 / SBS / 굿모닝연예,https://www.youtube.com//watch?v=xL7-1G10xD8
1,1,원하는 신체 부위를 골라서 맞출 수 있는 '신급 저격술'의 경찰 특공대 [영화리뷰 ...,https://www.youtube.com//watch?v=WyLq0ptqJ5g
2,2,[CCM prime] 🎞영화 같은 하루를 보낼 당신을 위해 with 샘윤 | 나의 ...,https://www.youtube.com//watch?v=8jaB2ECDLmc
3,3,✔드라마 OST 🥰 영화 사운드 트랙 컬렉션 (광고 없음) 🥰 Best Korean...,https://www.youtube.com//watch?v=xZn5YbqGOs8
4,4,수많은 어린이들에게 트라우마를 남긴 역대급 공포 영화 (결말포함),https://www.youtube.com//watch?v=rk01_xA5Bzk


In [5]:
dataset.shape

(20, 3)

In [26]:
dataset_title = dataset.title
dataset_title

0          영화 '발신제한' 개봉 첫 주말 박스오피스 정상 올랐다 / SBS / 굿모닝연예
1     원하는 신체 부위를 골라서 맞출 수 있는 '신급 저격술'의 경찰 특공대 [영화리뷰 ...
2     [CCM prime] 🎞영화 같은 하루를 보낼 당신을 위해 with 샘윤 | 나의 ...
3     ✔드라마 OST 🥰 영화 사운드 트랙 컬렉션 (광고 없음) 🥰 Best Korean...
4                  수많은 어린이들에게 트라우마를 남긴 역대급 공포 영화 (결말포함)
5     상상조차 하기 어려운 기술발전을 이룬 외계 종족이 지구를 침략하면 벌어지는 일 ㅎㄷ...
6              (꿀잼보장) 세자매가 모두 하길 원하면 벌어지는 일 [영화리뷰 결말포함]
7                                자신의 삶이 영화지만 그 사실을 모른다면
8                             현실에서 구체화되는 미스터리 영화 이야기(2)
9             권태기가 찾아온 부부에게 ㅅㅅ 처방해준 명의는 누구? [영화리뷰/결말포함]
10              한국인에게 칭찬 받는 중국 영화. 유덕화 판빙빙의 무협 액션 블록버스터
11             온에어티비 실시간TV 무료TV ㅣ 지상파 케이블 영화 스포츠 정주행 채널
12    고질라 대 콩!! [영화리뷰 결말포함][HERO TV-고질라#45] Godzilla...
13    2200년 미래, 유전자 조작으로 무엇이든 가능한 세상이 되었지만 무서운 범죄가 생...
14                      좀비 바이러스로 인류가 멸망한 세상에서 살아가는 한 여자
15           (영화리뷰)이제껏 경험하지 못한 냉혹한 해결사의 복수 액션 스릴러(결말포함)
16                [주식강의] 대한전선.영흥.이화전기.두산인프라코어.영화금속.메디콕스
17    김유정 X 박해준 X 이성민 @ NETFLIX 넷플릭스 영화 '제8일의 밤'

In [36]:
import re
p = re.compile('[a-zA-Z0-9]')

dataset_title = dataset.title
#dataset_title = dataset_title.apply(lambda x: p.findall(x))
dataset_title

0          영화 '발신제한' 개봉 첫 주말 박스오피스 정상 올랐다 / SBS / 굿모닝연예
1     원하는 신체 부위를 골라서 맞출 수 있는 '신급 저격술'의 경찰 특공대 [영화리뷰 ...
2     [CCM prime] 🎞영화 같은 하루를 보낼 당신을 위해 with 샘윤 | 나의 ...
3     ✔드라마 OST 🥰 영화 사운드 트랙 컬렉션 (광고 없음) 🥰 Best Korean...
4                  수많은 어린이들에게 트라우마를 남긴 역대급 공포 영화 (결말포함)
5     상상조차 하기 어려운 기술발전을 이룬 외계 종족이 지구를 침략하면 벌어지는 일 ㅎㄷ...
6              (꿀잼보장) 세자매가 모두 하길 원하면 벌어지는 일 [영화리뷰 결말포함]
7                                자신의 삶이 영화지만 그 사실을 모른다면
8                             현실에서 구체화되는 미스터리 영화 이야기(2)
9             권태기가 찾아온 부부에게 ㅅㅅ 처방해준 명의는 누구? [영화리뷰/결말포함]
10              한국인에게 칭찬 받는 중국 영화. 유덕화 판빙빙의 무협 액션 블록버스터
11             온에어티비 실시간TV 무료TV ㅣ 지상파 케이블 영화 스포츠 정주행 채널
12    고질라 대 콩!! [영화리뷰 결말포함][HERO TV-고질라#45] Godzilla...
13    2200년 미래, 유전자 조작으로 무엇이든 가능한 세상이 되었지만 무서운 범죄가 생...
14                      좀비 바이러스로 인류가 멸망한 세상에서 살아가는 한 여자
15           (영화리뷰)이제껏 경험하지 못한 냉혹한 해결사의 복수 액션 스릴러(결말포함)
16                [주식강의] 대한전선.영흥.이화전기.두산인프라코어.영화금속.메디콕스
17    김유정 X 박해준 X 이성민 @ NETFLIX 넷플릭스 영화 '제8일의 밤'

In [7]:
dataset_title = dataset_title.apply(lambda x: x.split())
dataset_title

0     [영화, '발신제한', 개봉, 첫, 주말, 박스오피스, 정상, 올랐다, /, SBS...
1     [원하는, 신체, 부위를, 골라서, 맞출, 수, 있는, '신급, 저격술'의, 경찰,...
2     [[CCM, prime], 🎞영화, 같은, 하루를, 보낼, 당신을, 위해, with...
3     [✔드라마, OST, 🥰, 영화, 사운드, 트랙, 컬렉션, (광고, 없음), 🥰, ...
4         [수많은, 어린이들에게, 트라우마를, 남긴, 역대급, 공포, 영화, (결말포함)]
5     [상상조차, 하기, 어려운, 기술발전을, 이룬, 외계, 종족이, 지구를, 침략하면,...
6     [(꿀잼보장), 세자매가, 모두, 하길, 원하면, 벌어지는, 일, [영화리뷰, 결말...
7                         [자신의, 삶이, 영화지만, 그, 사실을, 모른다면]
8                       [현실에서, 구체화되는, 미스터리, 영화, 이야기(2)]
9     [권태기가, 찾아온, 부부에게, ㅅㅅ, 처방해준, 명의는, 누구?, [영화리뷰/결말...
10    [한국인에게, 칭찬, 받는, 중국, 영화., 유덕화, 판빙빙의, 무협, 액션, 블록...
11    [온에어티비, 실시간TV, 무료TV, ㅣ, 지상파, 케이블, 영화, 스포츠, 정주행...
12    [고질라, 대, 콩!!, [영화리뷰, 결말포함][HERO, TV-고질라#45], G...
13    [2200년, 미래,, 유전자, 조작으로, 무엇이든, 가능한, 세상이, 되었지만, ...
14             [좀비, 바이러스로, 인류가, 멸망한, 세상에서, 살아가는, 한, 여자]
15    [(영화리뷰)이제껏, 경험하지, 못한, 냉혹한, 해결사의, 복수, 액션, 스릴러(결...
16             [[주식강의], 대한전선.영흥.이화전기.두산인프라코어.영화금속.메디콕스]
17    [김유정, X, 박해준, X, 이성민, @, NETFLIX, 넷플릭스, 영화

In [8]:
dataset_list = list(dataset_title.values)
dataset_list

[['영화',
  "'발신제한'",
  '개봉',
  '첫',
  '주말',
  '박스오피스',
  '정상',
  '올랐다',
  '/',
  'SBS',
  '/',
  '굿모닝연예'],
 ['원하는',
  '신체',
  '부위를',
  '골라서',
  '맞출',
  '수',
  '있는',
  "'신급",
  "저격술'의",
  '경찰',
  '특공대',
  '[영화리뷰',
  '결말포함]'],
 ['[CCM',
  'prime]',
  '🎞영화',
  '같은',
  '하루를',
  '보낼',
  '당신을',
  '위해',
  'with',
  '샘윤',
  '|',
  '나의',
  '사랑하는',
  '책,',
  '나를',
  '지으신',
  '주님',
  '|',
  '라이브',
  '찬양'],
 ['✔드라마',
  'OST',
  '🥰',
  '영화',
  '사운드',
  '트랙',
  '컬렉션',
  '(광고',
  '없음)',
  '🥰',
  'Best',
  'Korean',
  'OST',
  'Collection',
  'HD'],
 ['수많은', '어린이들에게', '트라우마를', '남긴', '역대급', '공포', '영화', '(결말포함)'],
 ['상상조차',
  '하기',
  '어려운',
  '기술발전을',
  '이룬',
  '외계',
  '종족이',
  '지구를',
  '침략하면',
  '벌어지는',
  '일',
  'ㅎㄷㄷ…..',
  '[결말포함/영화리뷰]'],
 ['(꿀잼보장)', '세자매가', '모두', '하길', '원하면', '벌어지는', '일', '[영화리뷰', '결말포함]'],
 ['자신의', '삶이', '영화지만', '그', '사실을', '모른다면'],
 ['현실에서', '구체화되는', '미스터리', '영화', '이야기(2)'],
 ['권태기가', '찾아온', '부부에게', 'ㅅㅅ', '처방해준', '명의는', '누구?', '[영화리뷰/결말포함]'],
 ['한국인에게', '칭찬', '받는', '중국', '영화.', '유덕화

In [9]:
def getDataFrame(dataset):
  te = TransactionEncoder()
  arrMatrix = te.fit(dataset).transform(dataset)
  return pd.DataFrame(arrMatrix, columns=te.columns_)

In [10]:
df = getDataFrame(dataset_list)
df.head()

,'발신제한','신급,'제8일의,(The,(결말포함),(광고,(꿀잼보장),(영화리뷰)이제껏,-1,-2,-3,/,2021,2200년,8th,@,Best,Collection,Godzilla,HD,"Kong,",Korean,NETFLIX,Night),OST,SBS,TV-고질라#45],X,[CCM,[결말포함/영화리뷰],[영화리뷰,[영화리뷰/결말포함],[주식강의],[통통컬처],prime],vs,with,|,✔드라마,ㅅㅅ,...,일,있는,자신의,저격술'의,정상,정주행,제작보고회,조작으로,좀비,종족이,주님,주말,중국,지구를,지상파,지으신,찬양,찾아온,채널,"책,",처방해준,첫,침략하면,칭찬,컬렉션,케이블,콩!!,트라우마를,트랙,특공대,판빙빙의,하기,하길,하루를,한,한국인에게,해결사의,현실에서,🎞영화,🥰
0,True,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,True,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False
1,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,...,False,True,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,True,False,True,True,False,False,...,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,True,False
3,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True,True,False,True,False,True,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,True,False,False,False,False,False,False,False,False,False,False,True
4,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,False,True,False,False,False,False,False,False,False,False,False,False,False,False


In [17]:
itemset = apriori(df, min_support=0.15, use_colnames=True)
itemset
itemset_test = itemset.iloc[:100]
itemset_test

,support,itemsets
0,0.15,('제8일의)
1,0.15,((The)
2,0.15,(8th)
3,0.15,(@)
4,0.15,(NETFLIX)
...,...,...
95,0.15,"([통통컬처], 김유정)"
96,0.15,"(넷플릭스, [통통컬처])"
97,0.15,"([통통컬처], 박해준)"
98,0.15,"([통통컬처], 밤')"


In [18]:
ar = association_rules(itemset_test, metric='confidence', min_threshold=0.3)
ar.sort_values(['support', 'confidence', 'lift'], ascending=False).head(10)

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction
0,((The),('제8일의),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
1,('제8일의),((The),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
2,('제8일의),(8th),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
3,(8th),('제8일의),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
4,(@),('제8일의),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
5,('제8일의),(@),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
6,(NETFLIX),('제8일의),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
7,('제8일의),(NETFLIX),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
8,(Night)),('제8일의),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
9,('제8일의),(Night)),0.15,0.15,0.15,1.0,6.666667,0.1275,inf
